In [55]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import warnings
warnings.filterwarnings('ignore')

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

db_config = {
    "host": os.getenv("DB_HOST"),
    "database": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "port": int(os.getenv("DB_PORT", 5432)), 
    "client_encoding": "UTF8",
}

In [2]:
# DB 설정 정보
db_config = {
        "host": "localhost",
        "database": "cadastre",
        "user": "postgres",
        "password": "1062",
        "port": 5432,
        "client_encoding": "UTF8",
}

In [87]:
region_name ={
        "서울특별시": "seoul",
        "부산광역시": "busan",
        "대구광역시": "daegu",
        "인천광역시": "incheon",
        "광주광역시": "gwangju",
        "대전광역시": "daejeon",
        "울산광역시": "ulsan",
        "세종특별자치시": "sejong",
        "경기도": "gyeonggi",
        "충청북도": "chungcheongbuk",
        "충청남도": "chungcheongnam",
        "전라남도": "jeollanam",
        "경상북도": "gyeongsangbuk",
        "경상남도": "gyeongsangnam",
        "제주특별자치도": "jeju",
        "강원특별자치도": "gangwon",
        "전북특별자치도": "jeonbuk",
}

In [88]:
class OutlierFinder:
        def __init__(self, db_config):
                self.db_config = db_config
                self.conn = None
        
        def connect(self):
                self.conn = psycopg2.connect(**self.db_config)
                
        def close(self):
                if self.conn:
                        self.conn.close()

        def get_outliers(self, target_table, region_kr, region_table_name):
                """
                지적도 EPSG: 5186
                사업 EPSG: 5179 -> 5186
                """
                try:
                        if not self.conn or self.conn.closed:
                                self.connect()
                        
                        cur = self.conn.cursor()
                        
                        query = sql.SQL(r"""
                        SELECT
                            ogc_fid
                            , cad_id
                            , region_name
                            , target_area
                            , cadastre_area
                            , shape_dist
                            , shape_diff_pct
                            , wkb_geometry, cad_geometry
                            , CASE 
                                WHEN cad_id IS NULL THEN 'name_error'
                                WHEN area_difference >= 5 THEN 'area_error'
                                WHEN ABS(shape_diff_pct) >= 5 THEN 'point_error'
                                when gis_contains is FALSE then 'contains_error'
                            END AS type_name
                        FROM (
                            SELECT
                                frg.ogc_fid,
                                cadastre.gis_building_uid AS cad_id,
                                frg.region_name,
                                frg.gis_area      AS target_area,
                                cadastre.gis_area AS cadastre_area,
                                CASE
                                    WHEN frg.gis_area = 0 THEN 100
                                    ELSE (frg.gis_area - cadastre.gis_area) / frg.gis_area * 100 
                                END AS area_difference,
                                CASE 
                                    -- 매칭되는 지적도가 없거나(NULL), 면적이 0이거나, 면적 차이가 5% 이상이면 계산 안 함
                                    WHEN cadastre.gis_area IS NULL
                                         OR frg.gis_area = 0
                                         OR ABS((frg.gis_area - cadastre.gis_area) / frg.gis_area * 100) >= 5 
                                    THEN NULL 
                                    ELSE 
                                        ST_HausdorffDistance(frg.wkb_geometry, cadastre.geometry)
                                END AS shape_dist,
                                CASE 
                                    WHEN cadastre.gis_area IS NULL
                                         OR frg.gis_area = 0
                                         OR ABS((frg.gis_area - cadastre.gis_area) / frg.gis_area * 100) >= 5 
                                    THEN NULL
                                    ELSE
                                        ST_HausdorffDistance(frg.wkb_geometry, cadastre.geometry)
                                        / NULLIF(
                                            GREATEST(
                                                ST_Perimeter(frg.wkb_geometry),
                                                ST_Perimeter(cadastre.geometry)
                                            ), 0
                                        ) * 100
                                END AS shape_diff_pct,
                                case
                                	WHEN cadastre.gis_area IS null OR frg.gis_area = 0 then null
                                    when ((frg.gis_area - cadastre.gis_area) / frg.gis_area * 100) > -5 then null
                                	else ST_Contains(
                                		ST_Buffer(cadastre.geometry, 0.01), frg.wkb_geometry)  -- 허용 오차 0.01(1cm)
                                end as gis_contains ,
                                frg.wkb_geometry  AS wkb_geometry,
                                cadastre.geometry AS cad_geometry
                            FROM (
                                -- 사업 폴리곤 + 주소 정규화
                                SELECT
                                    o.ogc_fid,
                                    o.shape_area,
                                    o.gis_area,
                                    ST_Transform(o.wkb_geometry, 5186) AS wkb_geometry,  -- EPSG 5186으로 변환
                                    REPLACE(
                                        REPLACE(
                                            REGEXP_REPLACE(
                                                -- PNU 기반 dtadd 생성
                                                p.pnu_name || ' ' ||
                                                CASE
                                                    WHEN SUBSTRING(o.pnu_cd, 11, 1) = '2' THEN '산'
                                                    ELSE ''
                                                END || ' ' ||
                                                (SUBSTRING(o.pnu_cd, 12, 4)::int)::text || '-' ||
                                                (SUBSTRING(o.pnu_cd, 16, 4)::int)::text,
                                                '\([^)]*\)', '', 'g'
                                            ),
                                            ' ', ''
                                        ),
                                        '-0', ''
                                    ) AS region_name
                                FROM {target_tbl} AS o
                                LEFT JOIN pfregdat.pnu_cd AS p
                                    ON p.pnu_code = SUBSTRING(o.pnu_cd, 1, 10)
                                WHERE
                                    -- 생성된 주소 문자열에 충청남도 포함
                                    p.pnu_name LIKE '%'||{region_kr}||'%'
                                    AND ST_SRID(o.wkb_geometry) = 5179
                            ) frg
                            LEFT JOIN (
                                -- 지적도 폴리곤 + 지역명 정규화
                                SELECT
                                    gis_area,
                                    gis_perimeter,
                                    geometry,
                                    gis_building_uid,
                                    CASE
                                        WHEN special_zone_code LIKE '%산%' THEN
                                            REPLACE(
                                                REGEXP_REPLACE(legal_dong_name, '\([^)]*\)', '', 'g'),
                                                ' ', ''
                                            ) || '산' || legal_dong_code
                                        ELSE
                                            REPLACE(
                                                REGEXP_REPLACE(legal_dong_name, '\([^)]*\)', '', 'g'),
                                                ' ', ''
                                            ) || legal_dong_code
                                    END AS region_name
                                FROM public.{region_tbl}
                            ) cadastre
                                ON frg.region_name = cadastre.region_name
                        ) t
                        WHERE
                              area_difference >= 5
                           OR area_difference IS NULL
                           OR ABS(shape_diff_pct) >= 5
                           or gis_contains is false;
                        """).format(
                                target_tbl=sql.Identifier(target_table.split('.')[0], target_table.split('.')[1]),
                                region_kr = sql.Literal(region_kr),
                                region_tbl=sql.Identifier(region_table_name)
                        )
                        
                        
                        # 쿼리 실행 
                        cur.execute(query)
                        
                        rows = cur.fetchall()
                        colnames = [desc[0] for desc in cur.description]
                        df = pd.DataFrame(rows, columns=colnames)
                        df['region'] = region_kr
                    
                        cur.close()

                except Exception as e:
                        # 에러 메시지 상세 출력
                        print(f"🛑 쿼리 실행 중 오류 ({region_kr}): {e}")
                        
                return df

In [86]:
finder = OutlierFinder(db_config)
target_table_name = 'target.target_space' # 스키마.테이블
all_list = []

for region_kr, region_table_name in region_name.items():
    print(f"--- {region_kr}({region_table_name}) 검증 시작 ---")
    
    result = finder.get_outliers(target_table_name, region_kr, region_table_name)
    all_list.append(result)   # 여기서는 리스트에만 쌓기

    print(f"-> {len(result)}건의 오류 검출 완료.\n")

# 루프 끝난 뒤 한 번에 concat
all_df = pd.concat(all_list, axis=0, ignore_index=True)

all_df.to_excel('result.xlsx', index=False)

--- 서울특별시(seoul) 검증 시작 ---
-> 0건의 오류 검출 완료.

--- 부산광역시(busan) 검증 시작 ---
-> 4건의 오류 검출 완료.

--- 대구광역시(daegu) 검증 시작 ---
-> 53건의 오류 검출 완료.

--- 인천광역시(incheon) 검증 시작 ---
-> 6건의 오류 검출 완료.

--- 광주광역시(gwangju) 검증 시작 ---
-> 0건의 오류 검출 완료.

--- 대전광역시(daejeon) 검증 시작 ---
-> 13건의 오류 검출 완료.

--- 울산광역시(ulsan) 검증 시작 ---
-> 105건의 오류 검출 완료.

--- 세종특별자치시(sejong) 검증 시작 ---
-> 1건의 오류 검출 완료.

--- 경기도(gyeonggi) 검증 시작 ---
-> 823건의 오류 검출 완료.

--- 충청북도(chungcheongbuk) 검증 시작 ---
-> 1777건의 오류 검출 완료.

--- 충청남도(chungcheongnam) 검증 시작 ---
-> 3086건의 오류 검출 완료.

--- 전라남도(jeollanam) 검증 시작 ---
-> 1164건의 오류 검출 완료.

--- 경상북도(gyeongsangbuk) 검증 시작 ---
-> 1505건의 오류 검출 완료.

--- 경상남도(gyeongsangnam) 검증 시작 ---
-> 1987건의 오류 검출 완료.

--- 제주특별자치도(jeju) 검증 시작 ---
-> 0건의 오류 검출 완료.

--- 강원특별자치도(gangwon) 검증 시작 ---
-> 1556건의 오류 검출 완료.

--- 전라북도(jeonbuk) 검증 시작 ---
-> 0건의 오류 검출 완료.

